# **1. Library & Konfigurasi**

In [1]:
import numpy as np
import random
import copy
import math
from collections import defaultdict

NUM_BINARY_SLOTS = 15
NUM_CARD_SLOTS = 10

CARD_TO_ID = {
    'AND': 1,
    'OR': 2,
    'NAND': 3,
    'NOR': 4,
    'XOR': 5
}

ID_TO_CARD = {v: k for k, v in CARD_TO_ID.items()}
INITIAL_BINARY_INPUTS = [0, 1, 0, 1, 0]

# **2. Definisi Game Environment**

In [2]:
class GameState:
    def __init__(self, player1_target=1, player2_target=0):
        self.binary_slots = np.full(NUM_BINARY_SLOTS, -1)
        self.binary_slots[:len(INITIAL_BINARY_INPUTS)] = INITIAL_BINARY_INPUTS
        self.card_slots = np.zeros(NUM_CARD_SLOTS)
        self.player1_hand = list(CARD_TO_ID.values())
        self.player2_hand = list(CARD_TO_ID.values())
        self.player1_target = player1_target
        self.player2_target = player2_target
        self.current_player = 1

    def _get_input_binary_indices(self, slot_idx):
        if 0 <= slot_idx <= 3:
            return slot_idx, slot_idx + 1
        elif 4 <= slot_idx <= 6:
            base = 5
            offset = slot_idx - 4
            return base + offset, base + offset + 1
        elif 7 <= slot_idx <= 8:
            base = 9
            offset = slot_idx - 7
            return base + offset, base + offset + 1
        elif slot_idx == 9:
            return 12, 13

    def _get_output_binary_index(self, slot_idx):
        if 0 <= slot_idx <= 3:
            return 5 + slot_idx
        elif 4 <= slot_idx <= 6:
            return 9 + (slot_idx - 4)
        elif 7 <= slot_idx <= 8:
            return 12 + (slot_idx - 7)
        elif slot_idx == 9:
            return 14

    def get_valid_moves(self):
        moves = []
        hand = self.player1_hand if self.current_player == 1 else self.player2_hand
        empty_slots = [i for i, slot in enumerate(self.card_slots) if slot == 0]

        for slot in empty_slots:
            idx1, idx2 = self._get_input_binary_indices(slot)
            if self.binary_slots[idx1] != -1 and self.binary_slots[idx2] != -1:
                for card in hand:
                    moves.append({'slot': slot, 'card': card})
        return moves

    def apply_move(self, move):
        slot = move['slot']
        card = move['card']
        hand = self.player1_hand if self.current_player == 1 else self.player2_hand
        hand.remove(card)
        self.card_slots[slot] = card

        idx1, idx2 = self._get_input_binary_indices(slot)
        output_idx = self._get_output_binary_index(slot)
        input_val1 = self.binary_slots[idx1]
        input_val2 = self.binary_slots[idx2]

        self.binary_slots[output_idx] = self._calculate_logic(card, input_val1, input_val2)
        self.current_player = 2 if self.current_player == 1 else 1

    def _calculate_logic(self, card_id, a, b):
        if card_id == CARD_TO_ID['AND']:
            return int(a and b)
        elif card_id == CARD_TO_ID['OR']:
            return int(a or b)
        elif card_id == CARD_TO_ID['NAND']:
            return int(not (a and b))
        elif card_id == CARD_TO_ID['NOR']:
            return int(not (a or b))
        elif card_id == CARD_TO_ID['XOR']:
            return int(a != b)

    def is_terminal(self):
        return self.binary_slots[NUM_BINARY_SLOTS - 1] != -1 and len(self.get_valid_moves()) == 0

    def get_winner(self):
        final_value = self.binary_slots[NUM_BINARY_SLOTS - 1]
        if final_value == -1:
            return 0
        if final_value == self.player1_target:
            return 1
        elif final_value == self.player2_target:
            return 2
        else:
            return 0

    def copy(self):
        return copy.deepcopy(self)

# **3. Implementasi MCTS**

### **3.1. MCTS Node**

In [3]:
class MCTSNode:
    def __init__(self, state, parent=None, move=None):
        self.state = state
        self.parent = parent
        self.move = move
        self.children = []
        self.visits = 0
        self.wins = 0
        self.untried_moves = state.get_valid_moves()

    def is_fully_expanded(self):
        return len(self.untried_moves) == 0

    def best_child(self, c_param=1.41):
        choices_weights = [
            (child.wins / child.visits) + c_param * math.sqrt(2 * math.log(self.visits) / child.visits)
            for child in self.children
        ]
        return self.children[np.argmax(choices_weights)]

    def expand(self):
        move = self.untried_moves.pop(random.randint(0, len(self.untried_moves) - 1))
        next_state = self.state.copy()
        next_state.apply_move(move)
        child_node = MCTSNode(next_state, parent=self, move=move)
        self.children.append(child_node)
        return child_node

    def rollout(self):
        current_state = self.state.copy()

        while not current_state.is_terminal():
            valid_moves = current_state.get_valid_moves()
            if not valid_moves:
                break
            move = random.choice(valid_moves)
            current_state.apply_move(move)

        return current_state.get_winner()

    def backpropagate(self, result, player_perspective):
        self.visits += 1
        if result == player_perspective:
            self.wins += 1
        elif result == 0:
            self.wins += 0.5

        if self.parent:
            self.parent.backpropagate(result, player_perspective)

### **3.2. MCTS Agent**

In [4]:
class MCTSAgent:
    def __init__(self, num_simulations=1000, player_id=2):
        self.num_simulations = num_simulations
        self.player_id = player_id

    def select_move(self, game_state):
        root = MCTSNode(game_state.copy())

        for _ in range(self.num_simulations):
            node = root

            while node.is_fully_expanded() and node.children:
                node = node.best_child()

            if not node.state.is_terminal() and not node.is_fully_expanded():
                node = node.expand()

            winner = node.rollout()

            node.backpropagate(winner, self.player_id)

        best_child = max(root.children, key=lambda c: c.visits)
        return best_child.move

    def get_move_statistics(self, game_state):
        root = MCTSNode(game_state.copy())

        for _ in range(self.num_simulations):
            node = root

            while node.is_fully_expanded() and node.children:
                node = node.best_child()

            if not node.state.is_terminal() and not node.is_fully_expanded():
                node = node.expand()

            winner = node.rollout()
            node.backpropagate(winner, self.player_id)

        stats = []
        for child in root.children:
            win_rate = child.wins / child.visits if child.visits > 0 else 0
            stats.append({
                'move': child.move,
                'visits': child.visits,
                'wins': child.wins,
                'win_rate': win_rate,
                'card': ID_TO_CARD.get(child.move['card'], 'Unknown'),
                'slot': child.move['slot']
            })

        return sorted(stats, key=lambda x: x['visits'], reverse=True)

# **4. Sistem Evaluasi AI**

### **4.1. Fungsi Evaluasi: AI vs AI**

In [5]:
def evaluate_ai_performance(num_games=100, ai1_simulations=500, ai2_simulations=500, show_progress=True):
    ai1 = MCTSAgent(num_simulations=ai1_simulations, player_id=1)
    ai2 = MCTSAgent(num_simulations=ai2_simulations, player_id=2)

    results = {
        'ai1_wins': 0,
        'ai2_wins': 0,
        'draws': 0,
        'total_moves': [],
        'game_lengths': []
    }

    print(f"\n{'='*70}")
    print(f"EVALUASI AI: {num_games} games")
    print(f"AI1 (Target=1): {ai1_simulations} simulasi MCTS")
    print(f"AI2 (Target=0): {ai2_simulations} simulasi MCTS")
    print(f"{'='*70}\n")

    for game_num in range(num_games):
        if show_progress and (game_num + 1) % 10 == 0:
            print(f"Progress: {game_num + 1}/{num_games} games", end="\r")

        game = GameState(player1_target=1, player2_target=0)
        move_count = 0

        while not game.is_terminal():
            valid_moves = game.get_valid_moves()
            if not valid_moves:
                break

            if game.current_player == 1:
                move = ai1.select_move(game)
            else:
                move = ai2.select_move(game)

            game.apply_move(move)
            move_count += 1

        winner = game.get_winner()

        if winner == 1:
            results['ai1_wins'] += 1
        elif winner == 2:
            results['ai2_wins'] += 1
        else:
            results['draws'] += 1

        results['total_moves'].append(move_count)
        results['game_lengths'].append(move_count)

    if show_progress:
        print(f"Progress: {num_games}/{num_games} games - SELESAI!     ")

    results['ai1_win_rate'] = results['ai1_wins'] / num_games
    results['ai2_win_rate'] = results['ai2_wins'] / num_games
    results['draw_rate'] = results['draws'] / num_games
    results['avg_game_length'] = np.mean(results['game_lengths'])
    results['std_game_length'] = np.std(results['game_lengths'])
    results['min_game_length'] = np.min(results['game_lengths'])
    results['max_game_length'] = np.max(results['game_lengths'])

    return results

### **4.2. Fungsi Evaluasi: Komparasi Simulasi**

In [6]:
def compare_different_simulations(num_games_per_config=50):
    print(f"\n{'='*70}")
    print("🔬 EVALUASI KOMPARATIF: Pengaruh Jumlah Simulasi MCTS")
    print(f"{'='*70}\n")

    simulation_configs = [
        (100, 500),
        (300, 500),
        (500, 500),
        (500, 1000),
        (1000, 1000),
    ]

    comparison_results = []

    for ai1_sims, ai2_sims in simulation_configs:
        print(f"\n🎯 Testing: AI1({ai1_sims} sims) vs AI2({ai2_sims} sims)")
        results = evaluate_ai_performance(
            num_games=num_games_per_config,
            ai1_simulations=ai1_sims,
            ai2_simulations=ai2_sims,
            show_progress=True
        )

        comparison_results.append({
            'config': f"AI1({ai1_sims}) vs AI2({ai2_sims})",
            'ai1_sims': ai1_sims,
            'ai2_sims': ai2_sims,
            'ai1_win_rate': results['ai1_win_rate'],
            'ai2_win_rate': results['ai2_win_rate'],
            'draw_rate': results['draw_rate'],
            'avg_length': results['avg_game_length']
        })

    print(f"\n{'='*90}")
    print("📊 RINGKASAN KOMPARASI")
    print(f"{'='*90}")
    print(f"{'Config':<25} | {'AI1 Win':<10} | {'AI2 Win':<10} | {'Draw':<8} | {'Avg Moves':<10}")
    print(f"{'-'*90}")

    for res in comparison_results:
        print(f"{res['config']:<25} | {res['ai1_win_rate']:>8.1%} | {res['ai2_win_rate']:>8.1%} | "
              f"{res['draw_rate']:>6.1%} | {res['avg_length']:>10.2f}")

    print(f"{'='*90}\n")

    print("💡 INSIGHTS:")
    print("  • Semakin banyak simulasi MCTS → AI semakin kuat")
    print("  • Perbedaan simulasi yang besar → Win rate berbeda signifikan")
    print("  • Game lebih panjang → Kedua AI lebih hati-hati (banyak simulasi)")
    print()

    return comparison_results

### **4.3. Fungsi Evaluasi: Strategi Pembukaan**

In [7]:
def evaluate_opening_strategies(num_games=100):
    print(f"\n{'='*70}")
    print("🎲 EVALUASI STRATEGI PEMBUKAAN")
    print(f"{'='*70}\n")

    ai = MCTSAgent(num_simulations=500, player_id=2)

    opening_moves = defaultdict(int)
    opening_wins = defaultdict(int)

    for game_num in range(num_games):
        if (game_num + 1) % 20 == 0:
            print(f"Progress: {game_num + 1}/{num_games} games", end="\r")

        game = GameState(player1_target=1, player2_target=0)

        ai_temp = MCTSAgent(num_simulations=500, player_id=1)
        first_move = ai_temp.select_move(game)

        move_key = f"Slot {first_move['slot']} - {ID_TO_CARD[first_move['card']]}"
        opening_moves[move_key] += 1

        game.apply_move(first_move)

        while not game.is_terminal():
            valid_moves = game.get_valid_moves()
            if not valid_moves:
                break

            if game.current_player == 1:
                move = ai_temp.select_move(game)
            else:
                move = ai.select_move(game)

            game.apply_move(move)

        if game.get_winner() == 1:
            opening_wins[move_key] += 1

    print(f"Progress: {num_games}/{num_games} games - SELESAI!     \n")

    print(f"{'Opening Move':<20} | {'Frequency':<12} | {'Win Rate':<10}")
    print(f"{'-'*50}")

    sorted_openings = sorted(opening_moves.items(), key=lambda x: x[1], reverse=True)

    for move, freq in sorted_openings:
        wins = opening_wins[move]
        win_rate = wins / freq if freq > 0 else 0
        print(f"{move:<20} | {freq:>4} ({freq/num_games:>5.1%}) | {win_rate:>8.1%}")

    print(f"{'='*70}\n")

### **4.4. Fungsi Helper Tampilan & Menu Utama Evaluasi**

In [8]:
def display_evaluation_results(results):
    print(f"\n{'='*70}")
    print("📊 HASIL EVALUASI")
    print(f"{'='*70}\n")

    print("🏆 KEMENANGAN:")
    print(f"  AI1 (Target=1): {results['ai1_wins']} wins ({results['ai1_win_rate']:.1%})")
    print(f"  AI2 (Target=0): {results['ai2_wins']} wins ({results['ai2_win_rate']:.1%})")
    print(f"  Draw:           {results['draws']} games ({results['draw_rate']:.1%})")

    print(f"\n📈 STATISTIK GAME:")
    print(f"  Rata-rata panjang game: {results['avg_game_length']:.2f} moves")
    print(f"  Standar deviasi:        {results['std_game_length']:.2f} moves")
    print(f"  Game terpendek:         {results['min_game_length']} moves")
    print(f"  Game terpanjang:        {results['max_game_length']} moves")

    print(f"\n📊 VISUALISASI WIN RATE:")
    ai1_bar = "█" * int(results['ai1_win_rate'] * 50)
    ai2_bar = "█" * int(results['ai2_win_rate'] * 50)
    draw_bar = "█" * int(results['draw_rate'] * 50)

    print(f"  AI1: {ai1_bar} {results['ai1_win_rate']:.1%}")
    print(f"  AI2: {ai2_bar} {results['ai2_win_rate']:.1%}")
    print(f"  Draw: {draw_bar} {results['draw_rate']:.1%}")

    print(f"\n{'='*70}\n")

def run_full_evaluation():
    print("\n" + "🚀"*35)
    print("SISTEM EVALUASI LENGKAP - GERBANG LOGIKA AI")
    print("🚀"*35 + "\n")

    print("Pilih mode evaluasi:")
    print("1. Evaluasi Standar (100 games, AI vs AI balanced)")
    print("2. Evaluasi Komparatif (berbagai konfigurasi simulasi)")
    print("3. Evaluasi Strategi Pembukaan")
    print("4. Evaluasi Lengkap (semua di atas)")
    print("5. Kembali ke game interaktif")

    while True:
        choice = input("\nPilih (1-5): ").strip()

        if choice == '1':
            results = evaluate_ai_performance(num_games=100, ai1_simulations=500, ai2_simulations=500)
            display_evaluation_results(results)
            break
        elif choice == '2':
            compare_different_simulations(num_games_per_config=50)
            break
        elif choice == '3':
            evaluate_opening_strategies(num_games=100)
            break
        elif choice == '4':
            print("\n🔥 Memulai evaluasi lengkap... Ini akan memakan waktu beberapa menit.\n")

            print("\n" + "="*70)
            print("BAGIAN 1: EVALUASI STANDAR")
            print("="*70)
            results = evaluate_ai_performance(num_games=100, ai1_simulations=500, ai2_simulations=500)
            display_evaluation_results(results)

            print("\n" + "="*70)
            print("BAGIAN 2: EVALUASI KOMPARATIF")
            print("="*70)
            compare_different_simulations(num_games_per_config=50)

            print("\n" + "="*70)
            print("BAGIAN 3: EVALUASI STRATEGI PEMBUKAAN")
            print("="*70)
            evaluate_opening_strategies(num_games=100)

            print("\n✅ EVALUASI LENGKAP SELESAI!")
            break
        elif choice == '5':
            play_interactive_game()
            break
        else:
            print("❌ Pilih 1-5!")

# **5. Mode Game Interaktif (Player vs AI)**

### **5.1. Fungsi Helper Tampilan**

In [9]:
def display_game_board(game):
    print("\n" + "="*60)
    print("PAPAN PERMAINAN - Struktur Bracket 5-4-3-2-1")
    print("="*60)

    print("\nLevel 1:")
    print("Binary: ", end="")
    for i in range(5):
        val = game.binary_slots[i]
        print(f"[{val if val != -1 else '?'}]", end=" ")
    print()
    print("Cards:  ", end="")
    for i in range(4):
        card = game.card_slots[i]
        if card == 0:
            print("[ ]", end=" ")
        else:
            print(f"[{ID_TO_CARD[int(card)][:3]}]", end="")
    print()

    print("\nLevel 2:")
    print("Binary: ", end="")
    for i in range(5, 9):
        val = game.binary_slots[i]
        print(f"  [{val if val != -1 else '?'}]", end=" ")
    print()
    print("Cards:  ", end="")
    for i in range(4, 7):
        card = game.card_slots[i]
        if card == 0:
            print("  [ ]", end=" ")
        else:
            print(f"  [{ID_TO_CARD[int(card)][:3]}]", end="")
    print()

    print("\nLevel 3:")
    print("Binary: ", end="")
    for i in range(9, 12):
        val = game.binary_slots[i]
        print(f"    [{val if val != -1 else '?'}]", end=" ")
    print()
    print("Cards:  ", end="")
    for i in range(7, 9):
        card = game.card_slots[i]
        if card == 0:
            print("    [ ]", end=" ")
        else:
            print(f"    [{ID_TO_CARD[int(card)][:3]}]", end="")
    print()

    print("\nLevel 4:")
    print("Binary: ", end="")
    for i in range(12, 14):
        val = game.binary_slots[i]
        print(f"        [{val if val != -1 else '?'}]", end=" ")
    print()
    print("Cards:  ", end="")
    card = game.card_slots[9]
    if card == 0:
        print("        [ ]")
    else:
        print(f"        [{ID_TO_CARD[int(card)][:3]}]")

    print("\nLevel 5 (FINAL):")
    val = game.binary_slots[14]
    print(f"            [{val if val != -1 else '?'}]")

    print("="*60)

def display_player_hand(game):
    hand = game.player1_hand if game.current_player == 1 else game.player2_hand
    print(f"\nKartu Player {game.current_player}: ", end="")
    for card_id in hand:
        print(f"[{ID_TO_CARD[card_id]}]", end=" ")
    print()

### **5.2. Fungsi Helper Input Pemain**

In [10]:
def get_player_input(game, valid_moves):
    print("\n" + "-"*60)
    print(f"GILIRAN PLAYER {game.current_player}")
    print("-"*60)

    display_player_hand(game)

    print(f"\nAnda punya {len(valid_moves)} pilihan move:")
    print("\nNo. | Slot | Kartu | Input1 | Input2 | Output")
    print("-"*60)

    for i, move in enumerate(valid_moves):
        slot = move['slot']
        card = move['card']
        idx1, idx2 = game._get_input_binary_indices(slot)
        output_idx = game._get_output_binary_index(slot)

        val1 = game.binary_slots[idx1]
        val2 = game.binary_slots[idx2]

        if card == CARD_TO_ID['AND']:
            result = int(val1 and val2)
        elif card == CARD_TO_ID['OR']:
            result = int(val1 or val2)
        elif card == CARD_TO_ID['NAND']:
            result = int(not (val1 and val2))
        elif card == CARD_TO_ID['NOR']:
            result = int(not (val1 or val2))
        elif card == CARD_TO_ID['XOR']:
            result = int(val1 != val2)

        print(f"{i:3d} | {slot:4d} | {ID_TO_CARD[card]:5s} | {int(val1):6d} | {int(val2):6d} | {result:6d}")

    print("-"*60)

    while True:
        try:
            choice = input(f"\nPilih move (0-{len(valid_moves)-1}) atau 'q' untuk quit: ").strip()

            if choice.lower() == 'q':
                print("Game dibatalkan.")
                return None

            choice_idx = int(choice)
            if 0 <= choice_idx < len(valid_moves):
                return valid_moves[choice_idx]
            else:
                print(f"❌ Pilihan harus antara 0-{len(valid_moves)-1}")
        except ValueError:
            print("❌ Input tidak valid! Masukkan angka.")

### **5.3. Fungsi Utama Game Interaktif**

In [11]:
def play_interactive_game():
    print("\n" + "="*60)
    print("🎮 SELAMAT DATANG DI GERBANG LOGIKA 🎮")
    print("="*60)
    print("\nInput awal:", INITIAL_BINARY_INPUTS)
    print("\n📌 ATURAN:")
    print("  • Anda adalah Player 1 - Target: Biner akhir = 1")
    print("  • AI adalah Player 2 - Target: Biner akhir = 0")
    print("  • Setiap kartu logika hanya bisa dipakai 1x")
    print("  • Kartu: AND, OR, NAND, NOR, XOR")
    print("="*60)

    print("\nPilih tingkat kesulitan AI:")
    print("1. Mudah (10 simulasi)")
    print("2. Sedang (50 simulasi)")
    print("3. Sulit (100 simulasi)")
    print("4. Sangat Sulit (1000 simulasi)")

    while True:
        try:
            difficulty = input("\nPilih (1-4): ").strip()
            if difficulty == '1':
                num_sims = 10
                break
            elif difficulty == '2':
                num_sims = 50
                break
            elif difficulty == '3':
                num_sims = 100
                break
            elif difficulty == '4':
                num_sims = 1000
                break
            else:
                print("❌ Pilih 1-4!")
        except:
            print("❌ Input tidak valid!")

    game = GameState(player1_target=1, player2_target=0)
    mcts_ai = MCTSAgent(num_simulations=num_sims, player_id=2)

    move_count = 0

    while not game.is_terminal():
        display_game_board(game)

        valid_moves = game.get_valid_moves()

        if not valid_moves:
            print("\n⚠️ Tidak ada move yang valid lagi!")
            break

        if game.current_player == 1:
            selected_move = get_player_input(game, valid_moves)
            if selected_move is None:
                return

            print(f"\n✅ Anda memilih: Slot {selected_move['slot']} - {ID_TO_CARD[selected_move['card']]}")
            input("\nTekan Enter untuk lanjut...")
        else:
            print("\n" + "-"*60)
            print("🤖 GILIRAN AI")
            print("-"*60)
            print("\nAI sedang berpikir", end="")
            for _ in range(3):
                print(".", end="", flush=True)
                import time
                time.sleep(0.3)
            print()

            selected_move = mcts_ai.select_move(game)

            print(f"\n🎯 AI memilih: Slot {selected_move['slot']} - {ID_TO_CARD[selected_move['card']]}")

            print("\n📊 Top 3 pertimbangan AI:")
            stats = mcts_ai.get_move_statistics(game)
            for i, stat in enumerate(stats[:3], 1):
                print(f"  {i}. {stat['card']} di slot {stat['slot']}: "
                      f"{stat['win_rate']:.1%} win rate ({stat['visits']} visits)")

            input("\nTekan Enter untuk lanjut...")

        game.apply_move(selected_move)
        move_count += 1

    display_game_board(game)

    print("\n" + "="*60)
    print("🏁 GAME OVER! 🏁")
    print("="*60)
    print(f"\nTotal moves: {move_count}")
    print(f"Biner akhir: {game.binary_slots[NUM_BINARY_SLOTS - 1]}")

    winner = game.get_winner()

    if winner == 1:
        print("\n🎉🎉🎉 SELAMAT! ANDA MENANG! 🎉🎉🎉")
        print("Anda berhasil membuat biner akhir = 1!")
    elif winner == 2:
        print("\n🤖 AI MENANG!")
        print("AI berhasil membuat biner akhir = 0.")
    else:
        print("\n⚖️ DRAW!")

    print("="*60)

# **6. Jalankan Aplikasi**

Gunakan salah satu dari sel di bawah ini untuk menjalankan mode yang diinginkan.

### **Mode 1: Menjalankan Evaluasi AI**

Jalankan sel ini untuk menjalankan menu evaluasi dan melihat performa AI.

In [12]:
run_full_evaluation()


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
SISTEM EVALUASI LENGKAP - GERBANG LOGIKA AI
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

Pilih mode evaluasi:
1. Evaluasi Standar (100 games, AI vs AI balanced)
2. Evaluasi Komparatif (berbagai konfigurasi simulasi)
3. Evaluasi Strategi Pembukaan
4. Evaluasi Lengkap (semua di atas)
5. Kembali ke game interaktif
❌ Pilih 1-5!
❌ Pilih 1-5!
❌ Pilih 1-5!
❌ Pilih 1-5!

EVALUASI AI: 100 games
AI1 (Target=1): 500 simulasi MCTS
AI2 (Target=0): 500 simulasi MCTS

Progress: 100/100 games - SELESAI!     

📊 HASIL EVALUASI

🏆 KEMENANGAN:
  AI1 (Target=1): 4 wins (4.0%)
  AI2 (Target=0): 96 wins (96.0%)
  Draw:           0 games (0.0%)

📈 STATISTIK GAME:
  Rata-rata panjang game: 10.00 moves
  Standar deviasi:        0.00 moves
  Game terpendek:         10 moves
  Game terpanjang:        10 moves

📊 VISUALISASI WIN RATE:
  AI1: ██ 4.0%
  AI2: ████████████████████████████████████████████████ 96.0%
  Draw:  0.0%




### **Mode 2: Bermain Melawan AI**

Jalankan sel ini untuk bermain secara interaktif melawan AI.

In [13]:
play_interactive_game()


🎮 SELAMAT DATANG DI GERBANG LOGIKA 🎮

Input awal: [0, 1, 0, 1, 0]

📌 ATURAN:
  • Anda adalah Player 1 - Target: Biner akhir = 1
  • AI adalah Player 2 - Target: Biner akhir = 0
  • Setiap kartu logika hanya bisa dipakai 1x
  • Kartu: AND, OR, NAND, NOR, XOR

Pilih tingkat kesulitan AI:
1. Mudah (10 simulasi)
2. Sedang (50 simulasi)
3. Sulit (100 simulasi)
4. Sangat Sulit (1000 simulasi)

PAPAN PERMAINAN - Struktur Bracket 5-4-3-2-1

Level 1:
Binary: [0] [1] [0] [1] [0] 
Cards:  [ ] [ ] [ ] [ ] 

Level 2:
Binary:   [?]   [?]   [?]   [?] 
Cards:    [ ]   [ ]   [ ] 

Level 3:
Binary:     [?]     [?]     [?] 
Cards:      [ ]     [ ] 

Level 4:
Binary:         [?]         [?] 
Cards:          [ ]

Level 5 (FINAL):
            [?]

------------------------------------------------------------
GILIRAN PLAYER 1
------------------------------------------------------------

Kartu Player 1: [AND] [OR] [NAND] [NOR] [XOR] 

Anda punya 20 pilihan move:

No. | Slot | Kartu | Input1 | Input2 | Output
